In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
from analysis_paper.file_browser_ import get_models
from config_path import GCP_RESULTS_PATH, PLOTS_PATH
from analysis.stat_utils import score_ci

In [12]:
from os.path import join, exists
from os import listdir
import pandas as pd
from sklearn import metrics
import numpy as np
from scipy import stats

In [5]:
def filter_model_dirs(Task, tuned, models, size=None):
    if size is None:
        size = ['base', 'NA']
    else:
        size= size+["NA"]
    print (size)
    if type(tuned) == list:
        tuned.append('NA')
    else:
        tuned = [tuned, 'NA']

    response_dirs = all_dirs[all_dirs.Task == Task]
    response_dirs = response_dirs[response_dirs.Model.isin(models)].copy()
    response_dirs = response_dirs[response_dirs.Tuned.isin(tuned)]
    response_dirs = response_dirs[response_dirs.Size.isin(size)]
    print(response_dirs)
    print(response_dirs[['Frozen', 'Model', 'Size', 'Task', 'Tuned', 'classifier']])
    # dirs = response_dirs.file
    # print (dirs)
    return response_dirs

In [6]:
def read_predictions(dirs_df):
    model_dict={}
    for i, row in dirs_df.iterrows():
        dir_ = row.file
        # model = row.Model + '_' +row.Size
        model = row.Model
        dir_ = join(GCP_RESULTS_PATH, dir_)
        prediction_file = [join(dir_,f) for f in listdir(dir_) if '0_testing.csv' in f][0]
        pred_df = pd.read_csv(prediction_file)
        print(pred_df.shape)
        print(pred_df.head())
        model_dict[model] = pred_df
    return model_dict

In [7]:
task='response'
models = ['BERT', 'CNN', 'TF-IDF']
all_dirs= get_models()
dirs_df = filter_model_dirs(Task=task, tuned=True, models=models)
model_dict = read_predictions(dirs_df)

['base', 'NA']
   Frozen   Model  Size      Task Tuned classifier  \
8      NA     CNN    NA  response    NA        CNN   
15   True    BERT  base  response  True        CNN   
18     NA  TF-IDF    NA  response    NA        CNN   

                                                 file  
8   ./updated_labels/JAMA/response_one_split_sizes...  
15  ./updated_labels/tuned_bert_cnn_frozen/respons...  
18  ./updated_labels/tfidf/response_one_split_tfid...  
   Frozen   Model  Size      Task Tuned classifier
8      NA     CNN    NA  response    NA        CNN
15   True    BERT  base  response  True        CNN
18     NA  TF-IDF    NA  response    NA        CNN
(1543, 7)
   Unnamed: 0  DFCI_MRN ehr_scan_date              PROC_DESCR  pred  \
0          60  118571.0    2010-06-23  ct chest with contrast   0.0   
1          61  118571.0    2010-11-15  ct chest with contrast   0.0   
2          62  118571.0    2011-03-11  ct chest with contrast   0.0   
3          63  118571.0    2011-07-22  ct ches

In [23]:
all_scores=[]
names=[]
xs=[]
avg_scores=[]
ci_uppers=[]
ci_lowers=[]
for i, k in enumerate(model_dict.keys()):
    df = model_dict[k]
    y_test = df['y']
    y_pred_score = df['pred_score']
    score, ci_lower, ci_upper, scores = score_ci(y_test, y_pred_score, score_fun=metrics.roc_auc_score,n_bootstraps=2000, seed=123)
    all_scores.append(scores)
    names.append(k)
    ci_lowers.append(ci_lower)
    ci_uppers.append(ci_upper)
    xs.append(np.random.normal(i + 1, 0.04, len(scores)))
    avg_scores.append(score)

In [19]:
model_dict.keys()

['CNN', 'BERT', 'TF-IDF']

In [25]:
all_scores_dict = dict(zip(names,all_scores))
ci_lowers_dict= dict(zip(names, ci_lowers))
ci_uppers_dict= dict(zip(names, ci_uppers))
avg_scores_dict = dict(zip(names, avg_scores))

In [21]:
all_scores_dict.keys()

['CNN', 'BERT', 'TF-IDF']

In [26]:
ci_lowers_dict

{'BERT': 0.9014420769747576,
 'CNN': 0.8868917748917748,
 'TF-IDF': 0.8799734188430868}

In [27]:
ci_uppers_dict

{'BERT': 0.9468831168831169,
 'CNN': 0.9390244975150637,
 'TF-IDF': 0.93550892021244}

In [28]:
avg_scores_dict

{'BERT': 0.9252129574601862,
 'CNN': 0.9137145465664107,
 'TF-IDF': 0.9079395019939193}

In [37]:
import itertools

for c1, c2 in itertools.combinations(all_scores_dict.keys(), 2):
    scores1, scores2 = all_scores_dict[c1], all_scores_dict[c2]
    twosample_results = stats.ttest_ind(scores1, scores2, equal_var=True)
    pvalue = twosample_results[1]
    print 'comparing {} ,  {}, pvalue={} '.format(c1, c2, pvalue)
    

comparing CNN ,  BERT, pvalue=3.85310144457e-152 
comparing CNN ,  TF-IDF, pvalue=2.10169366776e-44 
comparing BERT ,  TF-IDF, pvalue=1.37168334573e-316 


In [16]:
avg_scores

[0.9137145465664107, 0.9252129574601862, 0.9079395019939193]

In [33]:
stats.ttest_ind(scores1, scores1)

Ttest_indResult(statistic=0.0, pvalue=1.0)